<h2>Generalization case with k calibrations and 2 job types</h2>
Consider a problem of scheduling certain number of jobs that can have 2 possible execution times, on a single machine that needs occasional calibration. Once calibrated, machine stays in calibrated state for some interval of time that can take k possible lengths. Jobs cannot be executed unless the machine is in calibrated state.

<h3> Step 1: Initialization </h3>

In [1]:
from functools import lru_cache
import random, pprint


pp = pprint.PrettyPrinter()


job_types = range(2)
cal_types=[]
k=2
cal_len=[]
job_nb=[]
job_len=[]
cost_cal=0
cost_prob=0

#automatic or manual input
inp = int(input('Please enter 1 for user input, and 0 for automatically generated input '))
if(inp==1):
    j1 = int(input('Please input the length of the shorter job '))
    j2=int(input('Please input the length of the longer job '))
    job_len.extend((j1,j2))
    n1 = int(input('Please input the number of the shorter jobs '))
    n2=int(input('Please input the number of the longer jobs '))
    job_nb.extend((n1,n2))
    job_nb=tuple(job_nb)
    k=int(input('Please input the number of calibrations '))
    cal_types = range(k) 
    for i in cal_types:
        c = int(input('Please input the length of the calibration numbered ' + str(i+1)+' '))
        cal_len.append(c)
    cal_len.sort()
    cal_len=tuple(cal_len)
    cost_cal=int(input('Please input the calibration cost '))
    cost_prob=int(input('Please input the probing cost '))
else:   
    job_nb = tuple(random.randint(10,20) for _ in job_types)			# has to be a tuple for lru_cache
    job_len = [random.randint(10, 15) + 5 * i for i in job_types]
    k = random.randint(3,6)
    cal_types = range(k) 
    cal_len = [random.randint (30,40) + 15 * i for i in cal_types]
    cal_len.sort()  # just make sure the shortest one has index 0
    cal_len=tuple(cal_len)
    cost_cal = random.randint(100,200)
    cost_prob = random.randint(30, 90)

<h3> Step 2: Maximal packings </h3>
Compute for each calibration type, vectors of numbers of jobs that would fit in an interval of its length. Vectors are maximal in the sense that incrementing any of the job numbers would exceed the interval capacity

In [2]:
def fit_max(cal_len,job_len): #maximal packing function
    max_fit = [[] for ell in cal_types]
    for j, ell in enumerate(cal_len):
        for i0 in range(ell // job_len[0] + 1): 	# +1 because we want to include this last value
            i1 = (ell - i0 * job_len[0]) // job_len[1]
            max_fit[j].append([i0, i1])
    return max_fit

max_fit = fit_max(cal_len,job_len)
print("Maximal Coverings of calibrations")
pp.pprint(max_fit)

Maximal Coverings of calibrations
[[[0, 1], [1, 1], [2, 0]],
 [[0, 3], [1, 2], [2, 1], [3, 0], [4, 0]],
 [[0, 3], [1, 3], [2, 2], [3, 1], [4, 0], [5, 0]],
 [[0, 5], [1, 4], [2, 3], [3, 2], [4, 2], [5, 1], [6, 0]]]


<h3> Step 3: Scheduling functions - proposed, random, alternative schedule

In [3]:
@lru_cache(maxsize=None) 
def schedule(remaining, cost_cal, cost_prob):
	
	"""
	Solves optimally the scheduling problem with the given remaining
	number of jobs of each type.

	Returns a couple (exp_cost, sample_sch), where exp_cost is the expected
	cost of the produced schedule and sample_sch is the description  of
	one scenario of this schedule. The later consiste in a list of
	couples (technically 2-element lists). Every couple [t, v] describes
	one calibration interval. If t == -1, then the interval is not probed,
	and otherwise it is probed and t is its type. The vector v describes
	how many jobs of each type are packed into this interval.
	"""
    
	if sum(remaining) == 0:
		# no jobs to schedule
		return (0, [])

	# compute for each calibration type the schedule optimizing the expected cost
	opt_sch = [(float('inf'), []) for _ in cal_types]
	for i in cal_types:       
		# an alternative is of the form [e, s], 
		# where e is the expected cost, 
		# and s a sampled schedule with the restriction 
		# that the first interval is of type i.
		# try all possibilities to pack jobs into this first interval
		for u in max_fit[i]:
			# v0 is the actual packing vector, taking into account availabilities
			v0 = [min(remaining[j], u[j]) for j in job_types]
			if sum(v0):			# ignore empty packings
				v1 = tuple(remaining[j] - v0[j] for j in job_types)
				# schedule optimally v1, the vector of remaining jobs, once v0 is packed
				cost, sch = schedule(v1, cost_cal, cost_prob)
				alt = (cost_cal + cost, [[i, v0]] + sch)	# consider this alternative
				if alt < opt_sch[i]:
					opt_sch[i] = alt
	
	# Compare 2 alternatives

	
	# alternative 1: probe
	# compute expected cost, by averaging 
	exp_cost = cost_prob + sum(opt_sch[i][0] for i in cal_types) / len(cal_types)
	# compare with alternative 2: don't probe, use calibration type 0
	if exp_cost < opt_sch[0][0]:	
		# sample one scenario
		i = random.choice(cal_types)
		return (exp_cost, opt_sch[i][1])
	else:
		# alternative 2: don't probe, use type 0 interval
		
		# 0=calibration type, 1=schedule, 0=first interval, 0=type of this interval
		opt_sch[0][1][0][0] = -1 # indicate that we don't probe; shortest calibration, value for the first job type = -1
		# it is ok to modify this object as it is not used in another context
		return opt_sch[0]

In [4]:
#similar to the first scheduler but uses the second smallest instead of smallest intercval for the case of not probing
@lru_cache(maxsize=None) 
def schedule_2nd(remaining, cost_cal, cost_prob):

    
	if sum(remaining) == 0:
		# no jobs to schedule
		return (0, [])

	# compute for each calibration type the schedule optimizing the expected cost
	opt_sch = [(float('inf'), []) for _ in cal_types]
	for i in cal_types:       
		# an alternative is of the form [e, s], 
		# where e is the expected cost, 
		# and s a sampled schedule with the restriction 
		# that the first interval is of type i.
		# try all possibilities to pack jobs into this first interval
		for u in max_fit[i]:
			# v0 is the actual packing vector, taking into account availabilities
			v0 = [min(remaining[j], u[j]) for j in job_types]
			if sum(v0):			# ignore empty packings
				v1 = tuple(remaining[j] - v0[j] for j in job_types)
				# schedule optimally v1, the vector of remaining jobs, once v0 is packed
				cost, sch = schedule_2nd(v1, cost_cal, cost_prob)
				alt = (cost_cal + cost, [[i, v0]] + sch)	# consider this alternative
				if alt < opt_sch[i]:
					opt_sch[i] = alt
	
	# Compare 2 alternatives

	
	# alternative 1: probe
	# compute expected cost, by averaging 
	exp_cost = cost_prob + sum(opt_sch[i][0] for i in cal_types) / len(cal_types)
	i = random.choice(cal_types)
	# compare with alternative 2: don't probe, use calibration type 0
	if exp_cost < opt_sch[1][0]:	
		# sample one scenario
		return (exp_cost, opt_sch[i][1])
	else:
		cost = opt_sch[1][0]
		# alternative 2: don't probe,
		#don'tprobe. asuume type 1 int; but if generated int was type 1, then we add penalty in the value of probing cost
		if(i==0):
			cost=cost+cost_prob*3
		# 1=calibration type, 1=schedule, 0=first interval, 0=type of this interval
		opt_sch[1][1][0][0] = -1 # indicate that we don't probe; 2ndshortest calibration, value for the cal type = -1
		# it is ok to modify this object as it is not used in another context
		return (cost,opt_sch[1][1])

In [5]:
#random schedule, similar to the previous one but selects alternatives randomly 
@lru_cache(maxsize=None) 
def schedule_rdn(remaining, cost_cal, cost_prob):
    
	if sum(remaining) == 0:
		# no jobs to schedule
		return (0, [])

	# compute for each calibration type the schedule optimizing the expected cost
	opt_sch = [(float('inf'), []) for _ in cal_types]
	for i in cal_types:       
		# an alternative is of the form [e, s], 
		# where e is the expected cost, 
		# and s a sampled schedule with the restriction 
		# that the first interval is of type i.
		# try all possibilities to pack jobs into this first interval
		for u in max_fit[i]:
			# v0 is the actual packing vector, taking into account availabilities
			v0 = [min(remaining[j], u[j]) for j in job_types]
			if sum(v0):			# ignore empty packings
				v1 = tuple(remaining[j] - v0[j] for j in job_types)
				# schedule optimally v1, the vector of remaining jobs, once v0 is packed
				cost, sch = schedule_rdn(v1, cost_cal, cost_prob)
				alt = (cost_cal + cost, [[i, v0]] + sch)	# consider this alternative
				if alt < opt_sch[i]:
					opt_sch[i] = alt
	
	#  2 alternatives based on the value of choice that is random

	
	# alternative 1: probe
	# compute expected cost, by averaging
	choice = random.randint(0,1)
	exp_cost = cost_prob + sum(opt_sch[i][0] for i in cal_types) / len(cal_types)
    
	if choice==0:	
		# probing
		i = random.choice(cal_types)
		return (exp_cost, opt_sch[i][1])
	else:
		# alternative 2: don't probe, use type 0 interval
		
		# 0=calibration type, 1=schedule, 0=first interval, 0=type of this interval
		opt_sch[0][1][0][0] = -1 # indicate that we don't probe; shortest calibration, value for the first job type = -1
		# it is ok to modify this object as it is not used in another context
		return opt_sch[0]

<h3> Step 4: Representations of the schedules <h3>

In [6]:
print(f"instance {job_nb=} {job_len=} {cal_len=} {cost_cal=} {cost_prob=}")
print("\nProbing based on minimizing expected cost")
pp.pprint(schedule(job_nb, cost_cal, cost_prob))
print("\nProbing based on minimizing expected cost, 2nd smallest im case of not probing")
pp.pprint(schedule_2nd(job_nb, cost_cal, cost_prob))
print("\nRandomly probing")
pp.pprint(schedule_rdn(job_nb, cost_cal, cost_prob))

instance job_nb=(19, 10) job_len=[12, 16] cal_len=(30, 51, 60, 83) cost_cal=161 cost_prob=35

Probing based on minimizing expected cost
(1494.4762315936387,
 [[2, [5, 0]], [2, [1, 3]], [3, [4, 2]], [3, [4, 2]], [2, [1, 3]], [1, [4, 0]]])

Probing based on minimizing expected cost, 2nd smallest im case of not probing
(1369.182632446289,
 [[1, [0, 3]],
  [1, [4, 0]],
  [2, [1, 3]],
  [-1, [0, 3]],
  [-1, [2, 1]],
  [-1, [4, 0]],
  [-1, [4, 0]],
  [-1, [4, 0]]])

Randomly probing
(1629.6235172897577,
 [[-1, [2, 0]],
  [1, [0, 3]],
  [1, [2, 1]],
  [0, [1, 1]],
  [1, [2, 1]],
  [0, [0, 1]],
  [1, [3, 0]],
  [2, [5, 0]],
  [0, [0, 1]],
  [3, [4, 2]]])


<h3> Step 5: Performance comparisons between schedules <h3>
<h5> Comparing the performance of 121 pairs of schedules generated by two schedulers. Job numbers range from 10 to 20. Calibration and probing costs are randomly generated for each pair Job lengths and calibration lengths are fixed. Each entry in the output array refers to the percentage difference in costs between two schedulers.

<h5> 5.1: Proposed and random schedule </h5>

In [7]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#comparing the costs of random scheduler with proposed scheduler
#percentage difference in costs
def algo_rdn(sch1, sch2):
    result=[0]*2
    cost1 = sch1[0]
    cost2 = sch2[0]
    diff = (cost2-cost1)/cost2
    return diff*100
    

#plot 
z= []
for i in range(10,21):
    for j in range(10,21):   
        job_nb_ = tuple([i,j])
        c_c = random.randint(100,200)
        c_p = random.randint(20, 50)
        t = algo_rdn(schedule(job_nb_, c_c, c_p), schedule_2nd(job_nb_, c_c, c_p))
        z.append(t)
        
i = z.count(-1)
j=z.count(1)
z = np.array(z)
print(f"instance {job_len=} {cal_len=} ")
pp.pprint(z)

instance job_len=[12, 16] cal_len=(30, 51, 60, 83) 
array([ 1.23842219e-02, -9.11398963e+00, -7.32647428e+00, -1.30225765e+01,
       -2.05503961e+01, -1.00996327e+01, -1.06713570e+01, -5.75224691e+00,
       -1.02430198e+01, -1.04997769e+01, -1.24347233e+01, -5.61053367e+00,
       -7.81503817e+00, -1.50616806e+01, -1.34038630e+01, -7.87245121e+00,
       -6.02648774e+00, -8.46877943e+00, -8.01703687e+00, -1.04481865e+01,
       -8.63579590e+00, -7.28222881e+00, -9.67600989e+00, -1.82552446e+01,
       -2.38866511e+01, -9.97598278e+00, -1.29764546e+01, -6.55272906e+00,
       -7.54554139e+00, -7.69233981e+00, -1.91227277e+01, -1.20567148e+01,
       -1.18838211e+01, -6.20457193e+00, -1.61808161e+01, -1.66031541e+01,
       -2.65643135e+00, -1.78926263e+01, -9.79361879e+00, -5.83406640e+00,
       -2.05327484e+01, -1.14797726e+01, -4.78560518e+00, -5.28066402e+00,
       -1.48803627e+01, -1.42450845e+01, -1.01764026e+01, -8.39279096e+00,
       -1.04986266e+01, -1.17202974e+01, -2.5863

<h5>5.2: Proposed and alternative schedule</h5>

In [9]:
#plot 
p= []
for i in range(10,21):
    for j in range(10,21):   
        job_nb_ = tuple([i,j])
        c_c = random.randint(100,200)
        c_p = random.randint(20, 50)
        t = algo_rdn(schedule(job_nb_, c_c, c_p), schedule_2nd(job_nb_, c_c, c_p))
        p.append(t)
        
i = p.count(-1)
j=p.count(1)
p = np.array(z)
print(f"instance {job_len=} {cal_len=} ")
pp.pprint(p)

instance job_len=[12, 16] cal_len=(30, 51, 60, 83) 
array([ 1.23842219e-02, -9.11398963e+00, -7.32647428e+00, -1.30225765e+01,
       -2.05503961e+01, -1.00996327e+01, -1.06713570e+01, -5.75224691e+00,
       -1.02430198e+01, -1.04997769e+01, -1.24347233e+01, -5.61053367e+00,
       -7.81503817e+00, -1.50616806e+01, -1.34038630e+01, -7.87245121e+00,
       -6.02648774e+00, -8.46877943e+00, -8.01703687e+00, -1.04481865e+01,
       -8.63579590e+00, -7.28222881e+00, -9.67600989e+00, -1.82552446e+01,
       -2.38866511e+01, -9.97598278e+00, -1.29764546e+01, -6.55272906e+00,
       -7.54554139e+00, -7.69233981e+00, -1.91227277e+01, -1.20567148e+01,
       -1.18838211e+01, -6.20457193e+00, -1.61808161e+01, -1.66031541e+01,
       -2.65643135e+00, -1.78926263e+01, -9.79361879e+00, -5.83406640e+00,
       -2.05327484e+01, -1.14797726e+01, -4.78560518e+00, -5.28066402e+00,
       -1.48803627e+01, -1.42450845e+01, -1.01764026e+01, -8.39279096e+00,
       -1.04986266e+01, -1.17202974e+01, -2.5863